<a href="https://colab.research.google.com/github/GursimranKaur12/Google-Play-Store-Analytics.ipynb/blob/main/Google_Play_Store_Analytics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get install -y xvfb # Install Xvfb
!apt-get install -y xvfb # Install Xvfb
!pip install pyvirtualdisplay

from pyvirtualdisplay import Display
import tkinter as tk
from tkinter import ttk
from tkinter import messagebox
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error , r2_score
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error , r2_score
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk
import os

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libfontenc1 libxfont2 libxkbfile1 x11-xkb-utils xfonts-base xfonts-encodings xfonts-utils
  xserver-common
The following NEW packages will be installed:
  libfontenc1 libxfont2 libxkbfile1 x11-xkb-utils xfonts-base xfonts-encodings xfonts-utils
  xserver-common xvfb
0 upgraded, 9 newly installed, 0 to remove and 30 not upgraded.
Need to get 7,817 kB of archives.
After this operation, 12.0 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 libfontenc1 amd64 1:1.1.4-1build3 [14.7 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 libxfont2 amd64 1:2.0.5-1build1 [94.5 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/main amd64 libxkbfile1 amd64 1:1.1.0-1build3 [71.8 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy/main amd64 x11-xkb-utils amd64 7.7+5build4 [172 kB]
Get:5 http://archiv

In [2]:
display = Display(visible=0, size=(1200, 800)) # Create a virtual display with size 1200x800
display.start()

In [3]:
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


True

In [4]:
apps_df = pd.read_csv('Play Store Data.csv')
reviews_df = pd.read_csv('User Reviews.csv')

In [5]:
apps_df = apps_df.dropna(subset = ['Rating'])
for column in apps_df.columns:
  apps_df[column].fillna(apps_df[column].mode()[0], inplace =True)
apps_df.drop_duplicates(inplace=True)
apps_df = apps_df[apps_df['Rating'] <= 5]
reviews_df.dropna(subset=['Translated_Review'], inplace=True)

<ipython-input-5-4acb2292139c>:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  apps_df[column].fillna(apps_df[column].mode()[0], inplace =True)
<ipython-input-5-4acb2292139c>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  apps_df[column].fillna(apps_df[column].mode()[0], inplace =True)
<ipython-input-5-4acb2292139c>:4: SettingWithCopyWarnin

In [6]:
merged_df = pd.merge(apps_df, reviews_df, on='App', how='inner')


In [7]:
apps_df['Reviews'] = apps_df['Reviews'].astype(int)
apps_df['Installs'] = apps_df['Installs'] = apps_df['Installs'].str.replace(',' , '').str.replace('+', '').astype(int)
apps_df['Price'] = apps_df['Price'].str.replace('$', '').astype(float)

In [8]:
def convert_size(size):
  if'M' in size:
    return float(size.replace('M', ''))
  elif 'K' in size:
    return float(size.replace('K', ''))/1024
  else:
    return np.nan


In [9]:
apps_df['Size'] = apps_df['Size'].apply(convert_size)

In [10]:
apps_df['Log_Installs'] = np.log(apps_df['Installs'])
apps_df['Log_Reviews'] = np.log(apps_df['Reviews'])


In [11]:
def rating_group(rating):
  if rating >=4:
    return 'Top rated'
  elif rating >=3:
    return 'Above average'
  elif rating >=2:
    return 'Average'
  else:
    return 'Below average'

apps_df['Rating_Group'] = apps_df['Rating'].apply(rating_group)

In [12]:
apps_df['Revenue'] = apps_df['Price'] * apps_df['Installs']

In [13]:
sia = SentimentIntensityAnalyzer()
reviews_df['Sentiment_Score'] = reviews_df['Translated_Review'].apply(lambda x: sia.polarity_scores(str(x))['compound'])

In [14]:
apps_df['Last Updated'] = pd.to_datetime(apps_df['Last Updated'], errors = 'coerce')
apps_df['Year'] = apps_df['Last Updated'].dt.year

In [ ]:
class AppDashboard(tk.Tk):
  def __init__(self):
    super().__init__()
    self.title("Google Play Store Analysis Dashboard")
    self.geometry("1200*800")
    self.configure(bg='lightgray')


    main_frame = tk.Frame(self)
    main_frame.pack(fill = tk.Both, expand = True)

    canvas = tk.canvas(main_frame, bg = 'lightgray')
    v_scrollbar = ttk.Scrollbar(main_frame, orient = "vertical", command=canvas.yview)
    h_scrollbar = ttk.scrollbar(main_frame, orient="vertical", command =canvas.xview)

    v_scrollbar.pack(side = tk.RIGHT, fill = "Y")
    h_scrollbar.pack(side = tk.BOTTOM, fill = "X")

    canvas.pack(side="left", fill="both", expand=True)
    scrollable_frame = ttk.Frame(canvas)

    scrollable_frame.bind(
        "<Configure>",
        lambda e: canvas.configure(
            scrollregion = canvas.bbox("all")
        )
    )

    canvas.create_window((0,0), window= scrollable_frame, anchor="nw")
    canvas.configure(yscrollcommand = v_scrollbar.set, xscrollcommand=h_scrollbar.set)
    for i in range(6):
      scrollable_frame.columnconfigure(i, weight=1)

    self.create_category_analysis(scrollable_frame, 0, 0)
    self.create_type_analysis(scrollable_frame, 0, 1)
    self.create_rating_analysis(scrollable_frame, 0, 2)
    self.create_revenue_analysis(scrollable_frame, 0, 3)
    self.create_sentiment_analysis(scrollable_frame, 0, 4)
    self.create_ml_model_evaluation(scrollable_frame, 0, 5)
def create_category_analysis(self, parent, row, col):
  frame = ttk.Frame(parent)
  frame.grid(row=row, column=col, pady=5, sticky="nsew")
  frame.columnconfigure(0, weight=1)

  category_counts = apps_df['Category'].value_counts().nlargest(10)
  fig, ax = plt.subplots(figsize=(5,3))
  sns.barplot(x=category_counts.values, y=category_counts.index, palette='Spectural', ax=ax)
  ax.set_title('Top Categories on Play Store')
  ax.set_xlabel('Number of Apps')
  ax.set_ylabel('Category')
  fig.tight_layout()
  canvas = FigureCanvasTkAgg(fig, master=frame)
  canvas.draw()
  canvas.get_tk_widget().pack(fill=tk.BOTH, expand=True)

def create_type_analysis(self, parent, row, col):
  frame = ttk.Frame(parent, padding = "5")
  frame.grid(row=row, column=col, pady=5, sticky="nsew")
  frame.columnconfigure(0, weight=1)

  type_counts = apps_df['Type'].value_counts()
  fig, ax = plt.subplots(figsize=(3,2))
  ax.pie(type_counts, labels = type_counts.index, autopct='%1.1f%%', startangle=90, colors=sns.color_palette('pastel'))
  ax.set_title('Distribution of free and paid Apps')
  fig.tight_layout()
  canvas = FigureCanvasTkAgg(fig, master=frame)
  canvas.draw()
  canvas.get_tk_widget().pack(fill=tk.BOTH, expand=True)

def create_rating_sentiment_analysis(self, parent, row, col):
  frame = ttk.Frame(parent, padding="5")
  frame.grid(row=row, column=col, pady=5, sticky="nsew")
  frame.columnconfigure(0, weight=1)

  fig, ax = plt.subplots(figsize=(5,3))
  sns.histplot(apps_df['Rating'], bins=30, kde=True, color='purple', ax=ax)
  ax.set_title('Rating Distribution')
  ax.set_xlabel('Rating')
  ax.set_ylabel('Frequency')
  fig.tight_layout()
  canvas = FigureCanvasTkAgg(fig, master=frame)
  canvas.draw()
  canvas.get_tk_widget().pack(fill=tk.BOTH, expand=True)

  sentiment_counts = reviews_df['Sentiment_score'].apply(lambda x: 'Positive' if x >= 0.05 else ('Negative' if x <= -0.05 else 'Neutral')).value_counts()
  fig, ax = plt.subplots(figsize=(3,2))
  ax.pie(sentiment_counts, labels=sentiment_counts.index, autopct='%1.1f%%', startangle=90, colors=sns.color_palette('pastel'))
  ax.set_title('Sentiment Distribution')
  fig.tight_layout()
  canvas = FigureCanvasTkAgg(fig, master=frame)
  canvas.draw()
  canvas.get_tk_widget().pack(fill=tk.BOTH, expand=True)
def create_installaton_update_analysis(self, parent, row, col):
  frame = ttk.Frame(parent, padding="5")
  frame.grid(row=row, column=col, pady=5, sticky="nsew")
  frame.columnconfigure(0, weight=1)

  installs_by_category = apps_df.groupby('Category')['Installs'].sum().nlargest(10)
  fig, ax = plt.subplots(figsize=(5,3))
  sns.barplot(x=installs_by_category.values, y=installs_by_category.index, palette='viridis', ax=ax)
  ax.set_title('Install ations by Category')
  ax.set_xlabel('Number of Installs')
  ax.set_ylabel('Category')
  fig.tight_layout()
  canvas = FigureCanvasTkAgg(fig, master=frame)
  canvas.draw()
class AppDashboard(tk.Tk):
  def __init__(self):
    super().__init__()
    self.title("Google Play Store Analysis Dashboard")
    self.geometry("1200x800")
    self.configure(bg='lightgray')


    main_frame = tk.Frame(self)
    main_frame.pack(fill = tk.BOTH, expand = True)

    canvas = tk.Canvas(main_frame, bg = 'lightgray')
    v_scrollbar = ttk.Scrollbar(main_frame, orient = "vertical", command=canvas.yview)
    h_scrollbar = ttk.Scrollbar(main_frame, orient="horizontal", command =canvas.xview)

    v_scrollbar.pack(side = tk.RIGHT, fill = "y")
    h_scrollbar.pack(side = tk.BOTTOM, fill = "x")

    canvas.pack(side="left", fill="both", expand=True)
    scrollable_frame = ttk.Frame(canvas)

    scrollable_frame.bind(
        "<Configure>",
        lambda e: canvas.configure(
            scrollregion = canvas.bbox("all")
        )
    )

    canvas.create_window((0,0), window= scrollable_frame, anchor="nw")
    canvas.configure(yscrollcommand = v_scrollbar.set, xscrollcommand=h_scrollbar.set)
    for i in range(6):
      scrollable_frame.columnconfigure(i, weight=1)

    self.create_category_analysis(scrollable_frame, 0, 0)
    self.create_type_analysis(scrollable_frame, 0, 1)
    self.create_rating_analysis(scrollable_frame, 0, 2)
    self.create_revenue_analysis(scrollable_frame, 0, 3)
    self.create_sentiment_analysis(scrollable_frame, 0, 4)
    self.create_ml_model_evaluation(scrollable_frame, 0, 5)

  def create_category_analysis(self, parent, row, col):
    frame = ttk.Frame(parent)
    frame.grid(row=row, column=col, pady=5, sticky="nsew")
    frame.columnconfigure(0, weight=1)

    category_counts = apps_df['Category'].value_counts().nlargest(10)
    fig, ax = plt.subplots(figsize=(5,3))
    sns.barplot(x=category_counts.values, y=category_counts.index, palette='Spectral', ax=ax)
    ax.set_title('Top Categories on Play Store')
    ax.set_xlabel('Number of Apps')
    ax.set_ylabel('Category')
    fig.tight_layout()
    canvas = FigureCanvasTkAgg(fig, master=frame)
    canvas.draw()
    canvas.get_tk_widget().pack(fill=tk.BOTH, expand=True)

  def create_type_analysis(self, parent, row, col):
    frame = ttk.Frame(parent, padding = "5")
    frame.grid(row=row, column=col, pady=5, sticky="nsew")
    frame.columnconfigure(0, weight=1)

    type_counts = apps_df['Type'].value_counts()
    fig, ax = plt.subplots(figsize=(3,2))
    ax.pie(type_counts, labels = type_counts.index, autopct='%1.1f%%', startangle=90, colors=sns.color_palette('pastel'))
    ax.set_title('Distribution of free and paid Apps')
    fig.tight_layout()
    canvas = FigureCanvasTkAgg(fig, master=frame)
    canvas.draw()
    canvas.get_tk_widget().pack(fill=tk.BOTH, expand=True)

  def create_rating_analysis(self, parent, row, col): # Renamed for clarity and consistency
    frame = ttk.Frame(parent, padding="5")
    frame.grid(row=row, column=col, pady=5, sticky="nsew")
    frame.columnconfigure(0, weight=1)

    fig, ax = plt.subplots(figsize=(5,3))
    sns.histplot(apps_df['Rating'], bins=30, kde=True, color='purple', ax=ax)
    ax.set_title('Rating Distribution')
    ax.set_xlabel('Rating')
    ax.set_ylabel('Frequency')
    fig.tight_layout()
    canvas = FigureCanvasTkAgg(fig, master=frame)
    canvas.draw()
    canvas.get_tk_widget().pack(fill=tk.BOTH, expand=True)


  def create_revenue_analysis(self, parent, row, col): #Added function
      pass # Add your code for revenue analysis
  def create_sentiment_analysis(self, parent, row, col): #Added function
      pass # Add your code for sentiment analysis
  def create_ml_model_evaluation(self, parent, row, col):
    frame = ttk.Frame(parent, padding="5")
    frame.grid(row=row, column=col, pady=5, sticky="nsew")
    frame.columnconfigure(0, weight=1)

    x = apps_df[['Log_Reviews', 'Log_Installs', 'Price']]
    y = apps_df['Rating']

    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

    model = RandomForestRegressor(n_estimators=100, random_state=42)
    model.fit(x_train, y_train)

    y_pred = model.predict(x_test)
    ax.set_title(' Distribution of App Updates Over the Years')
    ax.set_xlabel('Year')
    ax.set_ylabel('Number of Updates')
    fig.tight_layout()
    canvas = FigureCanvasTkAgg(fig, master=frame)
    canvas.draw()
    canvas.get_tk_widget().pack(fill=tk.BOTH, expand=True)

def create_additional_insights(self, parent, row, col):
  frame = ttk.Frame(parent, padding="5")
  frame.grid(row=row, column=col, pady=5, sticky="nsew")
  frame.columnconfigure(0, weight=1)

  revenue_by_category = apps_df.groupby('Category')['Revenue'].sum().nlargest(10)
  fig, ax = plt.subplots(figsize=(5,3))
  sns.barplot(x=revenue_by_category.values, y=revenue_by_category.index, palette='viridis', ax=ax)
  ax.set_title('Revenue by Category')
  ax.set_xlabel('Revenue')
  ax.set_ylabel('Category')
  fig.tight_layout()
  canvas = FigureCanvasTkAgg(fig, master=frame)
  canvas.draw()
  canvas.get_tk_widget().pack(fill=tk.BOTH, expand=True)


  genres_counts = apps_df['Genres'].str.split(';').explode().value_counts().nlargest(10)
  fig, ax = plt.subplots(figsize=(5,3))
  sns.barplot(x=genres_counts.values, y=genres_counts.index, palette='viridis', ax=ax)
  ax.set_title('Top Genres on Play Store')
  ax.set_xlabel('Number of Apps')
  ax.set_ylabel('Genre')
  fig.tight_layout()
  canvas = FigureCanvasTkAgg(fig, master=frame)
  canvas.draw()
  canvas.get_tk_widget().pack(fill=tk.BOTH, expand=True)

  fig, ax = plt.subplots(figsize=(5,3))
  sns.boxplot(data=apps_df, x='Type', y ='Rating', palette='Set2', ax=ax)
  ax.set_title('Ratings for paid vs free apps')
  ax.set_xlabel('Type')
  ax.set_ylabel('Rating')
  fig.tight_layout()
  canvas = FigureCanvasTkAgg(fig, master=frame)
  canvas.draw()
  canvas.get_tk_widget().pack(fill=tk.BOTH, expand=True)
def create_ml_model_evaluation(self, parent, row, col):
    frame = ttk.Frame(parent, padding="5")
    frame.grid(row=row, column=col, pady=5, sticky="nsew")
    frame.columnconfigure(0, weight=1)

    x = apps_df[['Log_Reviews', 'Log_Installs', 'Price']]
    y = apps_df['Rating']

    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

    model = RandomForestRegressor(n_estimators=100, random_state=42)
    model.fit(x_train, y_train)

    y_pred = model.predict(x_test)

    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    fig, ax = plt.subplots(figsize=(5,3)) # Added this line to create fig and ax
    ax.scatter(y_test, y_pred, color='blue', alpha=0.3)
    ax.plot([0,5], [0,5], 'r--')
    ax.set_title(f'Ml Model Evaluation (MSE: {mse:.2f}, R2:{r2:.2f})')
    ax.set_xlabel('Actual Ratings')
    ax.set_ylabel('Predicted Ratings')
    fig.tight_layout()
    canvas = FigureCanvasTkAgg(fig, master=frame)
    canvas.draw()
import tkinter as tk
from tkinter import ttk
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

class AppDashboard(tk.Tk):
  def __init__(self):
    super().__init__()
    self.title("Google Play Store Analysis Dashboard")
    self.geometry("1200x800")
    self.configure(bg='lightgray')


    main_frame = tk.Frame(self)
    main_frame.pack(fill = tk.BOTH, expand = True)

    canvas = tk.Canvas(main_frame, bg = 'lightgray')
    v_scrollbar = ttk.Scrollbar(main_frame, orient = "vertical", command=canvas.yview)
    h_scrollbar = ttk.Scrollbar(main_frame, orient="horizontal", command =canvas.xview)

    v_scrollbar.pack(side = tk.RIGHT, fill = "y")
    h_scrollbar.pack(side = tk.BOTTOM, fill = "x")

    canvas.pack(side="left", fill="both", expand=True)
    scrollable_frame = ttk.Frame(canvas)

    scrollable_frame.bind(
        "<Configure>",
        lambda e: canvas.configure(
            scrollregion = canvas.bbox("all")
        )
    )

    canvas.create_window((0,0), window= scrollable_frame, anchor="nw")
    canvas.configure(yscrollcommand = v_scrollbar.set, xscrollcommand=h_scrollbar.set)
    for i in range(6):
      scrollable_frame.columnconfigure(i, weight=1)

    self.create_category_analysis(scrollable_frame, 0, 0)
    self.create_type_analysis(scrollable_frame, 0, 1)
    self.create_rating_analysis(scrollable_frame, 0, 2)
    self.create_revenue_analysis(scrollable_frame, 0, 3)
    self.create_sentiment_analysis(scrollable_frame, 0, 4)
    self.create_ml_model_evaluation(scrollable_frame, 0, 5)

  def create_category_analysis(self, parent, row, col):
    frame = ttk.Frame(parent)
    frame.grid(row=row, column=col, pady=5, sticky="nsew")
    frame.columnconfigure(0, weight=1)

    category_counts = apps_df['Category'].value_counts().nlargest(10)
    fig, ax = plt.subplots(figsize=(5,3))
    sns.barplot(x=category_counts.values, y=category_counts.index, palette='Spectral', ax=ax)
    ax.set_title('Top Categories on Play Store')
    ax.set_xlabel('Number of Apps')
    ax.set_ylabel('Category')
    fig.tight_layout()
    canvas = FigureCanvasTkAgg(fig, master=frame)
    canvas.draw()
    canvas.get_tk_widget().pack(fill=tk.BOTH, expand=True)

  def create_type_analysis(self, parent, row, col):
    frame = ttk.Frame(parent, padding = "5")
    frame.grid(row=row, column=col, pady=5, sticky="nsew")
    frame.columnconfigure(0, weight=1)

    type_counts = apps_df['Type'].value_counts()
    fig, ax = plt.subplots(figsize=(3,2))
    ax.pie(type_counts, labels = type_counts.index, autopct='%1.1f%%', startangle=90, colors=sns.color_palette('pastel'))
    ax.set_title('Distribution of free and paid Apps')
    fig.tight_layout()
    canvas = FigureCanvasTkAgg(fig, master=frame)
    canvas.draw()
    canvas.get_tk_widget().pack(fill=tk.BOTH, expand=True)

  def create_rating_analysis(self, parent, row, col): # Renamed for clarity and consistency
    frame = ttk.Frame(parent, padding="5")
    frame.grid(row=row, column=col, pady=5, sticky="nsew")
    frame.columnconfigure(0, weight=1)

    fig, ax = plt.subplots(figsize=(5,3))
    sns.histplot(apps_df['Rating'], bins=30, kde=True, color='purple', ax=ax)
    ax.set_title('Rating Distribution')
    ax.set_xlabel('Rating')
    ax.set_ylabel('Frequency')
    fig.tight_layout()
    canvas = FigureCanvasTkAgg(fig, master=frame)
    canvas.draw()
    canvas.get_tk_widget().pack(fill=tk.BOTH, expand=True)


  def create_revenue_analysis(self, parent, row, col): #Added function
      pass # Add your code for revenue analysis
  def create_sentiment_analysis(self, parent, row, col): #Added function
      pass # Add your code for sentiment analysis
  def create_ml_model_evaluation(self, parent, row, col):
    frame = ttk.Frame(parent, padding="5")
    frame.grid(row=row, column=col, pady=5, sticky="nsew")
    frame.columnconfigure(0, weight=1)

    x = apps_df[['Log_Reviews', 'Log_Installs', 'Price']]
    y = apps_df['Rating']

    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

    model = RandomForestRegressor(n_estimators=100, random_state=42)
    model.fit(x_train, y_train)

    y_pred = model.predict(x_test)

    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    fig, ax = plt.subplots(figsize=(5,3)) # Added this line to create fig and ax
    ax.scatter(y_test, y_pred, color='blue', alpha=0.3)
    ax.plot([0,5], [0,5], 'r--')
    ax.set_title(f'Ml Model Evaluation (MSE: {mse:.2f}, R2:{r2:.2f})')
    ax.set_xlabel('Actual Ratings')
    ax.set_ylabel('Predicted Ratings')
    fig.tight_layout()
    canvas = FigureCanvasTkAgg(fig, master=frame)
    canvas.draw()
    canvas.get_tk_widget().pack(fill=tk.BOTH, expand=True)

if __name__ == '__main__':
  app = AppDashboard()
  app.mainloop()



<ipython-input-15-ba11966f4bc0>:339: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.

  sns.barplot(x=category_counts.values, y=category_counts.index, palette='Spectral', ax=ax)
